## Importing modules

In [ ]:
import qiskit
from qiskit import QuantumCircuit, QuantumRegister, Aer, transpile, assemble
from qiskit.circuit.library import QFT
from qiskit.tools.jupyter import *
from qiskit.visualization import plot_histogram
from PIL import Image
import numpy as np

## Function to encode the image onto a quantum state

In [ ]:
def encode_image(image,n):
    
    im = Image.open(image,'r')
    im = im.convert("L")
    pixel_value = list(im.getdata())
    pixel_value_array = np.array(pixel_value)
    
    #Normalize the pixel values
    norm = np.linalg.norm(pixel_value_array)
    normalized_array = pixel_value_array/norm
    
    #Encode onto a quantum state
    qc = QuantumCircuit(n)
    qc.initialize(normalized_array.data, qc.qubits)
    
    return qc

## Function to apply quantum fourier transform on the encode image

In [ ]:
def apply_qft(qc,n):
    qc.append(QFT(n), qc.qubits)
    return qc

## Function to measure the image state

In [ ]:
def measurement_and_retrieval(qc,n):
    
    #Measure the state
    qc.measure_all()
    
    #Get the probability amplitudes
    aer_sim = Aer.get_backend('aer_simulator')
    shots = 4096
    t_qc = transpile(qc, aer_sim)
    qobj = assemble(t_qc, shots=shots)
    result = aer_sim.run(qobj).result()
    counts = result.get_counts(qc)
    fourier_transformed_array = np.zeros(2**n,dtype=float)
    bin_var = 0b0000
    dict_list = counts.keys()
    for i in range(2**n):
        str_var = bin(bin_var)[2:]
        str_var = str_var.zfill(n)
        if str_var in dict_list:
            fourier_transformed_array[i] = counts[f'{str_var}']/shots
            fourier_transformed_array[i] = np.sqrt(fourier_transformed_array[i])
        else:
            fourier_transformed_array[i] = 0
        bin_var += 1
    
    # Get the pixel values
    fourier_transformed_array = fourier_transformed_array*255
    transformed_pixel_data = np.zeros(2**n, dtype=int)
    for i in range(2**n): transformed_pixel_data[i] = int(fourier_transformed_array[i])
    picture_array = np.asarray([[0 for i in range(size)] for j in range(size)])
    for i in range(2**n):
        picture_array[i//size][i%size] = transformed_pixel_data[i]
    
    #Retrieval
    

## Function to retrieve the image

In [ ]:
def retrieve_image()
    picture_array = picture_array.astype(np.uint8)
    new_image = Image.fromarray(picture_array)
    new_image.save('new_image.png')
    
    return new

## Denoising

In [ ]:
def denoising(image,n):
    qc=encode_image(image,n)
    qc=apply_qft(qc,n)
    qft_pixels=measurement(qc,n)

    for i in range (length(qft_pixels)-no_of_cells,length(qft_pixels)):
    qft_pixels[i]=0

    new_image=Image.fromarray(qft_pixels)
    qc_iqft=encode_image(new_image,n)    
    qc_iqft.append(QFT(n,inverse=True), range(n))
    qc_iqft.barrier()
    qc_iqft.draw('mpl')
    picture_array=measurement(qc_iqft,n)
    new_image=retrieval(picture_array)

    return new_image